In [ ]:
!pip install numpy
!pip install matplotlib
!pip install ipywidgets
!pip install pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc
!pip install idaes-pse --pre
!idaes get-extensions --to ./solvers

In [ ]:
from scipy.special import gamma
from IPython.display import *
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pyomo.environ as pyo
import math
import sys
import os

os.environ['PATH'] += ':solvers'
opt_ipopt = pyo.SolverFactory('ipopt', executable='/content/solvers/ipopt')

In [ ]:


def plot_feasible_region(coef_x1, coef_x2, constraint1_value, constraint2_value, constraint3_value, constraint4_value, limit_integer, toggle_nonlinear_convex, toggle_nonlinear_nonconvex, solver, show_solution=False):


    if (solver_dropdown.value == 'GLPK'):
      print("GLPK (GNU Linear Programming Kit) is an open-source software package that solves linear programming (LP), mixed integer programming (MIP), and related optimization problems. GLPK includes a C-based library and a modeling language called GMPL (GNU Math Programming Language), which is similar to AMPL, making it easy to define and solve optimization models. Although it’s primarily designed for linear and integer optimization tasks, GLPK can be seamlessly integrated with Pyomo, a Python-based optimization modeling language, allowing users to define their models in Pyomo and solve them using GLPK. However, it\'s important to note that GLPK does not support nonlinear programming, so it is best suited for problems with linear relationships and discrete decision variables.")
    elif (solver_dropdown.value == 'CBC'):
      print('CLP (Coin-or Linear Programming) and CBC (Coin-or Branch and Cut) are open-source optimization solvers developed by the COIN-OR (Computational Infrastructure for Operations Research) project. CLP is a linear programming solver that efficiently handles large-scale linear problems, making it suitable for academic and industrial applications. CBC, on the other hand, is a mixed-integer programming (MIP) solver that builds on CLP\'s capabilities by incorporating branch-and-cut algorithms, enabling it to solve complex optimization problems involving both continuous and integer variables. Together, CLP and CBC provide a powerful and flexible solution for a wide range of linear and mixed-integer optimization tasks, and they can be easily integrated with other tools and programming languages, such as Python, via interfaces like PuLP and Pyomo.')
    elif (solver_dropdown.value == "IPOPT"):
      print('IPOPT (Interior Point OPTimizer) is a powerful open-source solver designed for large-scale nonlinear optimization problems. It specializes in solving nonlinear programming (NLP) problems, where both the objective function and constraints can be nonlinear. IPOPT employs an interior-point method, which is particularly effective for handling the challenges posed by complex, non-convex optimization landscapes. Widely used in academic research and industrial applications, IPOPT is known for its robustness and efficiency in solving a variety of optimization problems, including those in chemical engineering, economics, and machine learning. It can be seamlessly integrated with modeling environments like Pyomo and is supported by various programming interfaces, making it a versatile tool for advanced optimization tasks.')
    elif (solver_dropdown.value == "COUENNE"):
      print('COUENNE (Convex Over and Under ENvelopes for Nonlinear Estimation) is an open-source solver specifically designed for solving non-convex mixed-integer nonlinear programming (MINLP) problems. It employs a combination of global optimization techniques, including spatial branch-and-bound, to find globally optimal solutions for complex non-convex problems. COUENNE is particularly effective for problems where the objective function or constraints involve non-convexities, making it suitable for challenging optimization tasks in engineering, economics, and other fields requiring global optimization. As part of the COIN-OR project, COUENNE integrates well with modeling tools like Pyomo, providing a powerful and flexible tool for solving the most difficult MINLP problems.')
    elif (solver_dropdown.value == "BONMIN"):
      print('BONMIN (Basic Open-source Nonlinear Mixed INteger programming) is an open-source solver designed to tackle mixed-integer nonlinear programming (MINLP) problems. It combines both branch-and-bound and branch-and-cut strategies with nonlinear programming techniques, making it capable of solving optimization problems that involve both continuous and integer variables with nonlinear relationships. BONMIN is particularly useful in applications where the problem structure is complex and involves non-convexities, such as in engineering design, energy systems, and financial optimization. Developed as part of the COIN-OR project, BONMIN can be integrated with modeling tools like Pyomo, offering a flexible and robust solution for challenging MINLP problems.')

    fig, ax = plt.subplots(figsize=(8, 6))

    d = np.linspace(-0.5, 3.5, 300)
    x1, x2 = np.meshgrid(d, d)
    x = np.linspace(x1.min(), x1.max(), 2000)
    x21 = x + constraint1_value
    x22 = (constraint2_value - 8 * x) / 2.0
    objective_heatmap = np.fromfunction(lambda i, j: coef_x1 * i + coef_x2 * j, (300, 300), dtype=float)


    x1nl = 2 - (x - 1)**2 + constraint3_value
    x2nl = -1/2 + (x - 1)**2 - constraint4_value

    feasible_region = np.ones_like(x1, dtype=bool)
    feasible_region &= (x2 <= x1 + constraint1_value)
    feasible_region &= (8 * x1 + 2 * x2 <= constraint2_value)
    feasible_region &= (x1 >= 0)
    feasible_region &= (x2 >= 0)
    if toggle_nonlinear_convex:
        feasible_region &= ((x2-1)**2 <= 2-x1 + constraint3_value)
    if toggle_nonlinear_nonconvex:
        feasible_region &= ((x2-1)**2 >= x1+0.5 + constraint4_value)

    if show_solution:
      model = solve_model(1)
      ax.scatter([pyo.value(model[0])], [pyo.value(model[1])], color='cyan', label='Optimal solution')

    ax.imshow(feasible_region.astype(int) * objective_heatmap,
              extent=(x1.min(), x1.max(), x2.min(), x2.max()),
              origin="lower", cmap="OrRd", alpha=0.8)

    ax.plot(x, x21, label='x2 <= x1 + {}'.format(constraint1_value))
    ax.plot(x, x22, label='8x1 + 2x2 <= {}'.format(constraint2_value))
    ax.plot(np.zeros_like(x), x, label='x1 >= 0'.format(constraint3_value))
    ax.plot(x, np.zeros_like(x), label='x2 >= 0'.format(constraint4_value))

    if toggle_nonlinear_convex:
        ax.plot(x1nl, x, 'm', label=r'$(x_2-1)^2 \leq 2-x_1$')
    if toggle_nonlinear_nonconvex:
        ax.plot(x2nl, x, 'brown', label=r'$(x_2-1)^2 \geq x_1 + 1/2$')

    ax.set_xlim(x1.min(), x1.max())
    ax.set_ylim(x2.min(), x2.max())
    ax.legend(loc='upper right', prop={'size': 6})
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title("Feasible Region \n max z = {:.2f}x1 + {:.2f}x2".format(coef_x1, coef_x2))
    ax.grid(alpha=0.2)
    plt.show()

# Define the function that updates the dropdown based on the checkboxes
def update_solver_options(*args):
    options = ['GLPK', 'CBC', 'IPOPT', 'COUENNE', 'BONMIN']
    if limit_integer_checkbox.value:
        options.remove('IPOPT')
    if toggle_nonlinear_constraint_convex.value or toggle_nonlinear_constraint_nonconvex.value:
        options = [opt for opt in options if opt not in ('GLPK', 'CBC')]
    solver_dropdown.options = options
    if solver_dropdown.value not in options:
        solver_dropdown.value = options[0]

def solve_model(b):
    model = pyo.ConcreteModel()

    if limit_integer_checkbox.value:
        model.x1 = pyo.Var(within=pyo.Integers)
        model.x2 = pyo.Var(within=pyo.Integers)
    else:
        model.x1 = pyo.Var(within=pyo.NonNegativeReals)
        model.x2 = pyo.Var(within=pyo.NonNegativeReals)

    model.obj = pyo.Objective(expr=coef_x1_slider.value * model.x1 + coef_x2_slider.value * model.x2, sense=pyo.maximize)

    model.Constraint1 = pyo.Constraint(expr=model.x2 <= model.x1 + constraint1_value_slider.value)
    model.Constraint2 = pyo.Constraint(expr=8 * model.x1 + 2 * model.x2 <= constraint2_value_slider.value)
    model.Constraint3 = pyo.Constraint(expr=model.x1 >= 0)
    model.Constraint4 = pyo.Constraint(expr=model.x2 >= 0)

    if toggle_nonlinear_constraint_convex.value:
        model.Nonlinear1 = pyo.Constraint(expr=(model.x2-1)**2 <= 2 - model.x1 + constraint3_value_slider.value)
    if toggle_nonlinear_constraint_nonconvex.value:
        model.Nonlinear2 = pyo.Constraint(expr=(model.x2-1)**2 >= model.x1 + 0.5 + constraint4_value_slider.value)

    if (solver_dropdown.value == 'CBC'):
        opt = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
        result_obj = opt.solve(model, tee=False)
        return [model.x1, model.x2]
    elif (solver_dropdown.value == 'GLPK'):
        opt = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
        result_obj = opt.solve(model, tee=False)
        return [model.x1, model.x2]
    elif (solver_dropdown.value == 'IPOPT'):
        opt = pyo.SolverFactory('ipopt', executable='/content/solvers/ipopt')
        result_obj = opt.solve(model, tee=False)
        return [model.x1, model.x2]
    elif (solver_dropdown.value == 'COUENNE'):
        opt = pyo.SolverFactory('couenne', executable='/content/solvers/couenne')
        result_obj = opt.solve(model, tee=False)
        return [model.x1, model.x2]
    elif (solver_dropdown.value == 'BONMIN'):
        opt = pyo.SolverFactory('bonmin', executable='/content/solvers/bonmin')
        result_obj = opt.solve(model, tee=False)
        return [model.x1, model.x2]

coef_x1_slider = FloatSlider(value=5.5, min=0, max=10, step=0.1, description='Coefficient x1', layout=Layout(width='30%'), style={'description_width': 'initial'})
coef_x2_slider = FloatSlider(value=2.1, min=0, max=5, step=0.1, description='Coefficient x2', layout=Layout(width='30%'), style={'description_width': 'initial'})
constraint1_value_slider = FloatSlider(value=2, min=0, max=5, step=0.1, description='Constraint 1', layout=Layout(width='35%'))
constraint2_value_slider = FloatSlider(value=17, min=10, max=25, step=0.1, description='Constraint 2', layout=Layout(width='35%'))
constraint3_value_slider = FloatSlider(value=0, min=-5, max=5, step=0.1, description='Constraint 3', layout=Layout(width='35%'))
constraint4_value_slider = FloatSlider(value=0, min=-5, max=5, step=0.1, description='Constraint 4', layout=Layout(width='35%'))
show_solution_toggle = ToggleButton(value=False, description='Show Solution', tooltip='Show Solution', layout=Layout(width='18%'))

limit_integer_checkbox = Checkbox(value=False, description='Limit Integer Solution')
toggle_nonlinear_constraint_convex = Checkbox(value=False, description='Toggle Nonlinear Constraint Convex')
toggle_nonlinear_constraint_nonconvex = Checkbox(value=False, description='Toggle Nonlinear Constraint Nonconvex')

solver_dropdown = widgets.Dropdown(
    options=['GLPK', 'CBC', 'IPOPT', 'COUENNE', 'BONMIN'],
    value='GLPK',
    description='Solver:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)

limit_integer_checkbox.observe(update_solver_options, 'value')
toggle_nonlinear_constraint_convex.observe(update_solver_options, 'value')
toggle_nonlinear_constraint_nonconvex.observe(update_solver_options, 'value')

explanation = widgets.Label('')

out = interactive_output(plot_feasible_region, {
    'coef_x1': coef_x1_slider,
    'coef_x2': coef_x2_slider,
    'constraint1_value': constraint1_value_slider,
    'constraint2_value': constraint2_value_slider,
    'constraint3_value': constraint3_value_slider,
    'limit_integer': limit_integer_checkbox,
    'constraint4_value': constraint4_value_slider,
    'toggle_nonlinear_convex': toggle_nonlinear_constraint_convex,
    'toggle_nonlinear_nonconvex': toggle_nonlinear_constraint_nonconvex,
    'solver': solver_dropdown,
    'show_solution': show_solution_toggle
})

ui = VBox([
    HBox([limit_integer_checkbox, toggle_nonlinear_constraint_convex, toggle_nonlinear_constraint_nonconvex]),
    HBox([coef_x1_slider, coef_x2_slider]),
    HBox([constraint1_value_slider, constraint2_value_slider]),
    HBox([constraint3_value_slider, constraint4_value_slider]),
    HBox([solver_dropdown, show_solution_toggle]),
    HBox([explanation]),
    out
])

display(ui)
